In [1]:
import pandas as pd

**LOADING THE DATA**

In [2]:
orders=pd.read_csv("C:/Users/Lenovo/Projects/SQL_1/dataset/raw/olist_orders_dataset.csv")
items=pd.read_csv("C:/Users/Lenovo/Projects/SQL_1/dataset/raw/olist_order_items_dataset.csv")
payments=pd.read_csv("C:/Users/Lenovo/Projects/SQL_1/dataset/raw/olist_order_payments_dataset.csv")
customers=pd.read_csv("C:/Users/Lenovo/Projects/SQL_1/dataset/raw/olist_customers_dataset.csv")
products=pd.read_csv("C:/Users/Lenovo/Projects/SQL_1/dataset/raw/olist_products_dataset.csv")
categories=pd.read_csv("C:/Users/Lenovo/Projects/SQL_1/dataset/raw/product_category_name_translation.csv")
reviews=pd.read_csv("C:/Users/Lenovo/Projects/SQL_1/dataset/raw/olist_order_reviews_dataset.csv")

**COLUMN NAMES IN DATASET**

In [3]:
def columns():
    list=[orders,items,payments,customers,products,categories,reviews]
    for i in list:
        print(i.columns)

columns()

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date'],
      dtype='object')
Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value'],
      dtype='object')
Index(['order_id', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value'],
      dtype='object')
Index(['customer_id', 'customer_unique_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state'],
      dtype='object')
Index(['product_id', 'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm'],
      dtype='object')
Index(['product_category_name', 'product_category_name_english'], dtype='object')
Index(['review_id', 'order_id', 'revi

**MERGING THE TABLES**

In [4]:
df=orders.merge(items,on="order_id",how="inner")

In [5]:
df=df.merge(payments,on="order_id",how="left")

In [6]:
df=df.merge(customers,on="customer_id",how="left")

In [7]:
df=df.merge(products,on="product_id",how="left")

In [8]:
df=df.merge(categories,on="product_category_name",how="left")

In [9]:
df=df.merge(reviews,on='order_id',how='left')

**CLEANING**

In [10]:
df.shape

(118310, 37)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118310 entries, 0 to 118309
Data columns (total 37 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       118310 non-null  object 
 1   customer_id                    118310 non-null  object 
 2   order_status                   118310 non-null  object 
 3   order_purchase_timestamp       118310 non-null  object 
 4   order_approved_at              118295 non-null  object 
 5   order_delivered_carrier_date   117056 non-null  object 
 6   order_delivered_customer_date  115722 non-null  object 
 7   order_estimated_delivery_date  118310 non-null  object 
 8   order_item_id                  118310 non-null  int64  
 9   product_id                     118310 non-null  object 
 10  seller_id                      118310 non-null  object 
 11  shipping_limit_date            118310 non-null  object 
 12  price                         

In [12]:
df.isnull().sum().sort_values(ascending=False)

review_comment_title             104418
review_comment_message            68628
order_delivered_customer_date      2588
product_category_name_english      1734
product_category_name              1709
product_name_lenght                1709
product_photos_qty                 1709
product_description_lenght         1709
order_delivered_carrier_date       1254
review_answer_timestamp             978
review_id                           978
review_score                        978
review_creation_date                978
product_width_cm                     20
product_length_cm                    20
product_height_cm                    20
product_weight_g                     20
order_approved_at                    15
payment_type                          3
payment_sequential                    3
payment_value                         3
payment_installments                  3
order_status                          0
customer_id                           0
order_id                              0


Initial data quality checks identified missing delivery dates for undelivered orders.  It may be due to the orders not delivered or cancelled.
<br>Therefore,

In [13]:
df["order_delivered_customer_date"]=pd.to_datetime(
    df["order_delivered_customer_date"],
    errors="coerce")

Next, column which has more missing values is that product name in english field. These values can be handled by representing the product name as unknown quantity instead of dropping them.

In [14]:
df["product_category_name_english"].fillna("Unknown",inplace=True)
df["payment_type"].fillna("Unknown", inplace=True)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8872\1666318898.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["product_category_name_english"].fillna("Unknown",inplace=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8872\1666318898.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves a

In [15]:
drop_cols = [
    "order_approved_at",
    "order_delivered_carrier_date",
    "seller_id",
    "product_name_lenght",
    "product_description_lenght",
    "product_photos_qty"
]

df.drop(columns=drop_cols, inplace=True)


In [16]:
df.duplicated().sum()

np.int64(0)

In [17]:
date_cols=[
    "order_purchase_timestamp",
    "order_delivered_customer_date",
    "order_estimated_delivery_date"
]

In [18]:
for col in date_cols:
    df[col]=pd.to_datetime(df[col],errors="coerce")

In [19]:
df["payment_value"] = df["payment_value"].astype(float)
df["price"] = df["price"].astype(float)
df["freight_value"] = df["freight_value"].astype(float)

In [20]:
df["order_month"]=df["order_purchase_timestamp"].dt.month
df["order_year"]=df["order_purchase_timestamp"].dt.year
df["order_day"]=df["order_purchase_timestamp"].dt.day
df["order_weekday"]=df["order_purchase_timestamp"].dt.day_name()

In [21]:
df["delivery_time_days"]=(
    df["order_delivered_customer_date"]-df["order_purchase_timestamp"]
).dt.days

df["estimated_delivery_days"]=(
    df["order_estimated_delivery_date"]
    -df["order_purchase_timestamp"]
).dt.days
df["delivery_delay"]=df["delivery_time_days"]-df["estimated_delivery_days"]

In [22]:
#Create a new column of total cost of the product including freight charges
df["order_total"]=df["price"]+df["freight_value"]

#TOTAL REVENUE
total_revenue=df.groupby("order_id")["payment_value"].sum().reset_index()

In [23]:
df["customer_order_count"]=(
    df.groupby("customer_unique_id")["order_id"]
    .transform("nunique")
)

df["customer_type"]=df["customer_order_count"].apply(
    lambda x:"Repeat" if x >1 else "New"
)

In [24]:
df[["price","payment_value"]].describe()

,price,payment_value
count,118310.000000,118307.000000
mean,120.646603,172.575102
std,184.109691,267.110727
min,0.850000,0.000000
25%,39.900000,60.850000
50%,74.900000,108.200000
75%,134.900000,189.260000
max,6735.000000,13664.080000


In [25]:
upper_limit=df["price"].quantile(0.99)
df.loc[df["price"]>upper_limit,"price"]=upper_limit

In [26]:
df.isnull().sum().sort_values(ascending=False)
#df.describe()

review_comment_title             104418
review_comment_message            68628
delivery_time_days                 2588
delivery_delay                     2588
order_delivered_customer_date      2588
product_category_name              1709
review_creation_date                978
review_score                        978
review_answer_timestamp             978
review_id                           978
product_length_cm                    20
product_weight_g                     20
product_height_cm                    20
product_width_cm                     20
payment_sequential                    3
payment_value                         3
payment_installments                  3
shipping_limit_date                   0
product_id                            0
order_item_id                         0
order_estimated_delivery_date         0
order_purchase_timestamp              0
order_status                          0
customer_id                           0
order_id                              0


In [27]:
#Average order value
aov=df.groupby("order_id")["payment_value"].sum().mean()

In [28]:
#To find the top categories purchased
top_categories=(
    df.groupby("product_category_name_english")["payment_value"]
    .sum()
    .sort_values(ascending=False)
    .head(10)
)

In [29]:
df.to_csv(
    "C:/Users/Lenovo/Projects/SQL_1/dataset/processed/final_olist_orders_analysis.csv",
    index=False
)